In [ ]:
Chương trình tuần tự( 100, 1000, 5000, 10000, 20000)

In [ ]:
import numpy as np
import time

# Khởi tạo ma trận kích thước lớn
N = 20000 # Thay đổi kích thước N phù hợp với bộ nhớ
A = np.random.rand(N, N)
B = np.random.rand(N, N)

# Nhân ma trận tuần tự
start_time = time.time()
C = np.dot(A, B)
end_time = time.time()

print(f"Nhân ma trận tuần tự mất: {end_time - start_time:.4f} giây")


Nhân ma trận tuần tự mất: 287.1811 giây


Chương trình sử dụng thư viện cupy


In [ ]:
import cupy as cp
import time

# Khởi tạo ma trận kích thước lớn
N = 20000 # Thay đổi kích thước N phù hợp với GPU
A = cp.random.rand(N, N)
B = cp.random.rand(N, N)

# Nhân ma trận trên GPU
start_time = time.time()
C = cp.dot(A, B)
cp.cuda.Stream.null.synchronize()  # Đồng bộ để đảm bảo tính thời gian chính xác
end_time = time.time()

print(f"Nhân ma trận bằng CuPy mất: {end_time - start_time:.4f} giây")


Nhân ma trận bằng CuPy mất: 63.4141 giây


Sử dụng thư viện numba

In [ ]:
import numpy as np
from numba import cuda
import time

# Hàm nhân ma trận trên GPU
@cuda.jit
def matmul_gpu(A, B, C):
    row, col = cuda.grid(2)
    if row < C.shape[0] and col < C.shape[1]:
        temp = 0
        for k in range(A.shape[1]):
            temp += A[row, k] * B[k, col]
        C[row, col] = temp

# Khởi tạo ma trận kích thước lớn
N = 10000 # Thay đổi kích thước N phù hợp với GPU
A = np.random.rand(N, N).astype(np.float32)
B = np.random.rand(N, N).astype(np.float32)
C = np.zeros((N, N), dtype=np.float32)

# Copy dữ liệu lên GPU
A_gpu = cuda.to_device(A)
B_gpu = cuda.to_device(B)
C_gpu = cuda.to_device(C)
# Cấu hình lưới và khối
threads_per_block = (16, 16)
blocks_per_grid = (int(np.ceil(N / threads_per_block[0])), int(np.ceil(N / threads_per_block[1])))
# Nhân ma trận
start_time = time.time()
matmul_gpu[blocks_per_grid, threads_per_block](A_gpu, B_gpu, C_gpu)
cuda.synchronize()  # Đồng bộ để đảm bảo tính thời gian chính xác
end_time = time.time()
# Copy kết quả về CPU
C = C_gpu.copy_to_host()
print(f"Nhân ma trận bằng Numba mất: {end_time - start_time:.4f} giây")
